# Install Weaviate Python Module

In [ ]:
!pip install -U weaviate-client

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.7/293.7 kB 6.6 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.7/223.7 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 50.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 55.0 MB/s eta 0:00:00a 0:00:01


# Weaviate Python API Example

In [2]:
import requests 
import json
import weaviate
import weaviate.classes as wvc

print("Connecting to Weaviate Vector Database...")
client = weaviate.connect_to_local(
    host="10.0.1.89",
    port=8080,
    grpc_port=50051,
    additional_config=weaviate.config.AdditionalConfig(timeout=(15, 150))
)
try:
    print("Pulling blog posts from jasonacox.com...")
    resp = requests.get('https://jasonacox.com/wordpress/json')
    data = json.loads(resp.text)  # Load data

    # Load data
    question_objs = list()
    for i, d in enumerate(data["items"]):
        print(f"{i} - {d['title']}")
        question_objs.append({
            "title": d["title"],
            "url": d["url"],
            "content_text": d["content_text"],
        })

    # Delete collection
    try:
        client.collections.delete("jasonacox")
    except:
        pass

    # Create collection
    print("Creating collection...")
    questions = client.collections.create(
        name="jasonacox",
        vectorizer_config=wvc.config.Configure.Vectorizer.text2vec_transformers()
    )

    # Add data
    print("Vectorizing and storing blog posts...")
    questions.data.insert_many(question_objs)  # This uses batching under the hood

    # Query the Vector DB
    print("Searching database...")
    questions = client.collections.get("jasonacox")
    
    prompts = ["List some facts about solar.", "What are some tips on leadership?"]

    for query in prompts:
        response = questions.query.near_text(
            query=query,
            limit=5
        )
        print()
        print(query)
        print("------------------")
        for i in response.objects:
            print(f"{i.properties['title']} - {i.properties['url']}")

finally:
    client.close()  # Close client gracefully
    print()
    print("Done.")

Connecting to Weaviate Vector Database...
Pulling blog posts from jasonacox.com...
0 - Turn up the Temperature of your Thinking!
1 - Keep Growing
2 - Imagine 2029
3 - 12 Days of Christmas
4 - 23.5 Degrees
5 - My Space Camp Adventure
6 - Birthday Connections
7 - Traditions
8 - Feast of Life
9 - Christmas Lights
10 - The Spirit of AI
11 - Imagination Realm
12 - Memory
13 - Dreams
14 - 1202
15 - Chimney Sweeps and Chainsaws
16 - The Next Word
17 - Time
18 - Hurriquake
19 - The Unlimited Future
20 - Dog Days of Summer
21 - The Journey to AGI
22 - A Declaration of Happiness
23 - Storm Warning
24 - JasonGPT-1 : Adventures in AI
25 - Generative AI
26 - The Best Pottery
27 - A Seat of Opportunity
28 - Be a Stranger
29 - Moore’s Optimism
30 - The Art of Removal
31 - The Joy of Snow
32 - A Tribute to Game Changers
33 - Leading Change
34 - Listening
35 - The Present under the Tree
36 - Money Shot
37 - Holiday Hugs
38 - Thanksgiving
39 - Halloween!
40 - Listen
41 - Grid Bugs
42 - Investments Unlim